In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# configure gmaps
gmaps.configure(api_key=g_key)
print(g_key)

In [4]:
cities_df = pd.read_csv('../WeatherPy/ow_weather_df.csv')
cities_df.dropna()
cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Lebu,17,CL,1579819808,66,-37.62,-73.65,62.294,9.95465
1,Yellowknife,40,CA,1579819808,76,62.46,-114.35,-4.000,8.05320
2,Bluff,66,NZ,1579819809,64,-46.60,168.33,73.994,2.99758
3,Mar del Plata,40,AR,1579819809,55,-38.00,-57.56,89.006,17.22490
4,Aykhal,11,RU,1579819809,65,66.00,111.50,-38.200,6.71100


In [5]:
# store latitude and longitude in locations
locations = cities_df[['Lat','Lng']]
# store humidity as rating
rating = cities_df['Humidity'].astype(float)
# plot heatmap
fig = gmaps.figure()
# create heat layer
heat_layer = gmaps.heatmap_layer(locations,weights=rating,\
                                 dissipating=False,max_intensity=10,\
                                 point_radius=1)
# add layer
fig.add_layer(heat_layer)
# display figure
fig

Figure(layout=FigureLayout(height='420px'))